In [13]:
!pip install spacy==3.5.3
!pip install pandas==2.0.1
!pip install scikit-learn==1.2.2
!python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 206.0 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [15]:
script = pd.read_csv('Game_of_Thrones_Script_count_senti.csv')
script.head()

,Release Date,Season,Episode,Episode Title,Name,Sentence,Tokenize Words,Tokenize Words Alphanumeric Only,Sentence Word Count,Row,Family Name,Sentiment Scores,Polarity,Subjectivity
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...,"['What', 'do', 'you', 'expect', '?', 'They', ""...","['What', 'do', 'you', 'expect', 'They', 'savag...",25,0,other,"Sentiment(polarity=-0.125, subjectivity=0.375)",-0.125,0.375
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...,"['I', ""'ve"", 'never', 'seen', 'wildlings', 'do...","['I', 'never', 'seen', 'wildlings', 'do', 'a',...",21,1,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?,"['How', 'close', 'did', 'you', 'get', '?']","['How', 'close', 'did', 'you', 'get']",5,2,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.,"['Close', 'as', 'any', 'man', 'would', '.']","['Close', 'as', 'any', 'man', 'would']",5,3,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.,"['We', 'should', 'head', 'back', 'to', 'the', ...","['We', 'should', 'head', 'back', 'to', 'the', ...",7,4,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000


In [16]:
top_20 = script.groupby(['Name', 'Family Name'])[['Sentence Word Count']].sum().reset_index().sort_values(by=['Sentence Word Count']
                                                                                         , ascending=[0])

In [17]:
script_top5 = script[script["Name"].isin(top_20["Name"][:5])]

In [18]:
script_top_20 = script[script["Name"].isin(top_20["Name"][:20])]

In [19]:
script_top_20 = script_top_20[["Sentence", "Name"]]

In [20]:
script_top_20.columns = ["Sentence", "Label"]

In [21]:
script_top_20.head()

,Sentence,Label
15,Go on. Father's watching.,jon snow
16,And your mother.,jon snow
18,Thank you.,sansa stark
21,"Don't think too much, Bran.",jon snow
22,Relax your bow arm.,robb stark


In [22]:
script_top_20.to_csv('Game_of_Thrones_Script_top20.csv', encoding='utf-8', index=False)

In [23]:
script_top5["Name"].unique()

array(['jon snow', 'jaime lannister', 'cersei lannister',
       'tyrion lannister', 'daenerys targaryen'], dtype=object)

In [24]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [25]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [26]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))

In [27]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [28]:
from sklearn.model_selection import train_test_split

X = script_top5['Sentence'] # the features we want to analyze
ylabels = script_top5['Name'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Logistic Regression Classifier
classifier = LogisticRegression(max_iter=1000) 

# Create pipeline using Bag of Words
pipe = Pipeline([
    ("cleaner", predictors()),
    ('vectorizer', CountVectorizer()),
    ('classifier', classifier)
])

# model generation
pipe.fit(X_train, y_train)


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f032e48f670>),
                ('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression(max_iter=1000))])

In [30]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))


Logistic Regression Accuracy: 0.3829185520361991


In [31]:
pipe.classes_

array(['cersei lannister', 'daenerys targaryen', 'jaime lannister',
       'jon snow', 'tyrion lannister'], dtype=object)

In [32]:
pipe.predict_proba(["Yes, and the dwarf loves his whine and the women"])

array([[0.03723945, 0.06254509, 0.01468617, 0.1453793 , 0.74014998]])

In [33]:
pipe.predict_proba(["The high sparrow annoys me."])

array([[0.7704482 , 0.04285122, 0.06087351, 0.0682678 , 0.05755927]])

In [34]:
pipe.predict_proba(["I own the dragons."])

array([[0.23333601, 0.25861693, 0.10664684, 0.22663733, 0.17476288]])

In [35]:
pipe.predict_proba(["I can't fight anymore because I lost my hand."])

array([[0.03638479, 0.44195361, 0.04295454, 0.42997252, 0.04873453]])

In [36]:
pipe.predict_proba(["My father is Eddard Stark."])

array([[0.16601118, 0.03203128, 0.17894808, 0.20420871, 0.41880075]])

In [37]:
pipe.predict_proba(["My father is Tywin Lannister."])

array([[0.15890063, 0.06403182, 0.11603996, 0.24933321, 0.41169438]])

In [38]:
pipe.predict_proba(["The north remembers."])

array([[0.15317228, 0.21578007, 0.09263295, 0.37873266, 0.15968204]])

The last model was generic. In the next step I will try to tune the model a little and use Season 1 - 7 for train and cross validation. Then let's see how well this model performs on Season 8.

In [39]:
X = script_top5[script_top5["Season"] != "Season 8"]['Sentence'] # the features we want to analyze
ylabels = script_top5[script_top5["Season"] != "Season 8"]['Name'] # the labels, or answers, we want to test against

X_holdout = script_top5[script_top5["Season"] == "Season 8"]['Sentence']#
ylabels_holdout = script_top5[script_top5["Season"] == "Season 8"]['Name']

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Logistic Regression Classifier
classifier = LogisticRegression()

hyperparams = {'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], "classifier__solver": ["newton-cg", "liblinear"]}

# Create pipeline using Bag of Words
pipe2 = Pipeline([
    ("cleaner", predictors()),
    ('vectorizer', CountVectorizer()),
    ('classifier', classifier)
])

grid = GridSearchCV(pipe2, param_grid=hyperparams, cv=10)

# model generation
grid.fit(X, ylabels)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('cleaner',
                                        <__main__.predictors object at 0x7f031a6eb2b0>),
                                       ('vectorizer', CountVectorizer()),
                                       ('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'classifier__solver': ['newton-cg', 'liblinear']})

In [41]:
grid.best_params_

{'classifier__C': 0.1, 'classifier__solver': 'newton-cg'}

In [42]:
grid.best_score_

0.3528291186646217

In [43]:
best_model = grid.best_estimator_

In [44]:
from sklearn import metrics

# Predicting with my holdout dataset (Season 8)
predictions = best_model.predict(X_holdout)
print("Logistic Regression Accuracy:",metrics.accuracy_score(ylabels_holdout, predictions))

Logistic Regression Accuracy: 0.3451178451178451


In [45]:
best_model.classes_

array(['cersei lannister', 'daenerys targaryen', 'jaime lannister',
       'jon snow', 'tyrion lannister'], dtype=object)

In [46]:
best_model.predict_proba(["Yes, and the dwarf loves his whine and the women"])

array([[0.10444115, 0.14221143, 0.06164573, 0.16221139, 0.5294903 ]])

In [47]:
best_model.predict_proba(["The love for my children is above everything"])

array([[0.33146899, 0.17109578, 0.11233061, 0.14558726, 0.23951736]])

In [48]:
best_model.predict_proba(["I will break the chains and free millions of people."])

array([[0.07577507, 0.49322644, 0.06449631, 0.15950396, 0.20699823]])

In [49]:
best_model.predict_proba(["I won't be able to fight ever again..."])

array([[0.10546638, 0.15657447, 0.16440357, 0.33871338, 0.2348422 ]])

In [50]:
best_model.predict_proba(["My father is Tywin Lannister"])

array([[0.19927519, 0.11360641, 0.12751016, 0.19333597, 0.36627226]])